In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))
import pandas as pd
import numpy as np

'''Load models:'''
from whoosh.index import create_in
from whoosh.index import open_dir
from whoosh.fields import Schema, TEXT
from whoosh.qparser import MultifieldParser,OrGroup, query
from whoosh import scoring
from whoosh import highlight
import os, os.path

In [ ]:
df = pd.read_csv.(os.path.abspath(os.path.join('..', 'data/processed/'))+"/taxonomy_final.csv", index=False)

In [ ]:
'''create directory to save whoosh indeces:'''
if os.path.exists(os.path.abspath(os.path.join('whoosh'))):
    shutil.rmtree(os.path.abspath(os.path.join('whoosh')))
os.mkdir(os.path.abspath(os.path.join('whoosh')))

schema = Schema(PIMS_ID=TEXT(stored=True), 
                leading_country=TEXT(stored=True), 
                title=TEXT(stored=True), 
                all_text=TEXT(stored=True) ,
                all_text_clean=TEXT(stored=True), 
                all_text_clean_spacy=TEXT(stored=True))
ix = create_in(os.path.abspath(os.path.join('whoosh')), schema)
writer = ix.writer()

for i in df.index:                                         
    writer.add_document(PIMS_ID = str(df.loc[i,"PIMS_ID"]), 
                      leading_country = str(df.loc[i, "leading_country"]),
                      title = str(df.loc[i, "title"]),
                      all_text = str(df.loc[i, "all_text"]),
                      all_text_clean = str(df.loc[i, "all_text_clean"]),
                       all_text_clean_spacy = str(df.loc[i, "all_text_clean_spacy"])) 
writer.commit()

In [ ]:
from whoosh.index import open_dir
ix = open_dir(os.path.abspath(os.path.join('whoosh')))

In [ ]:
'''Set parameters and variables for search:'''
question = 'chemical and waste'
#question = 'How much was the chemical sector in Vietnam growing for the period 1990 – 2004?'
#question = 'In what substances are higher levels of chemicals of concern found?'


weighting_type = scoring.BM25F()
fields = ['all_text']
#allow_q = query.Term("Lead_Country", "Vietnam")
og = OrGroup.factory(0.9) #bonus scaler

'''Parse Search:'''
parser = MultifieldParser(fields, ix.schema, group = og)
q = parser.parse(question)

In [ ]:
pims = []
title = []
with ix.searcher(weighting = weighting_type) as s:
  results = s.search(q, limit = 10)
  results.fragmenter = highlight.SentenceFragmenter()
  results.formatter = highlight.UppercaseFormatter()
  print('--------------------------')
  print('Question:', question)
  print('--------------------------')
  print('--------------------------')
  print("Search Results for most relevant document - searched in", fields, ":")
  for res in results:
    pims.append(res['PIMS_ID'])
    title.append(res['title'])
    print('-------------')
    print('document:', res['PIMS_ID'], res['title'])
summary = dict(zip(pims, title))
summary = pd.DataFrame(summary.items(), columns=['PIMS_ID', 'title'])